# Stock-Duration-To-Come-Back-From-Crisis

In [210]:
import sys
sys.executable # Check in which virtual environment I am operating

'/Users/Armellini1/opt/anaconda3/envs/finenv/bin/python'

Generating a table with the worst crisis for a stock over a given period of time. And the time it took for the stock to come back to the initial price it had fallen from.

This will give an historical view of the kind of crises a company faced in its history, and the duration it took the company to recover form that crisis.


| Company   | Crisis     |  Low | from High | Change | got back by | after (years)
|-----------|----------- |------|-----------|--------|-------------|--------------
| Boeing    | 1973-03-10 | \$10 |  \$20     | -50%   |1976-09-10   |3.5

In [211]:
import numpy as np
import pandas as pd
import yfinance as yf # Module to retrieve data on financial instruments (similar to 'yahoo finance')
import matplotlib
from matplotlib import pyplot as plt # Import pyplot for plotting
from pandas.plotting import register_matplotlib_converters # to register Register Pandas Formatters and Converters with matplotlib.
from pandas.tseries.offsets import DateOffset
plt.style.use('seaborn') # using a specific matplotlib style
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

## Parameters

### `stock`

Captures the sock (company) to be studied. 

The variable should be set with the official "Ticker" of the company.

E.g: `stock = 'AAPL'` for Ticker AAPL (Ticker of the company Apple).

In [212]:
stock = 'BA' # stock ticker under study. Example: 'AAPL'
stock_name = 'Boeing' # name of the stock. Example: 'Apple'

### `period`

Period of time under study. 

This is the period of time during which we want to find the crisis for the stock under study.

For example, it can be the last 5 years, the last 10 years or the maximum period for which we have data.

In [213]:
period = 'max' # valid period values: 1y,2y,5y,10y,ytd,max

### `time_window`

The time window is the moving time interval used to calculate the loss of the sotck.
At each point in time, the loss will be calculated compared to a reference point dated at the beginning of that time window.

This way, the loss calculated will be representative of the loss over that rolling time window.

E.g.: 
`time_window = '20'`
The change will be calculated over a rolling time window of 20 trading days.


In [214]:
time_window = 20 # in open trading days

### `large_loss`

A stock will be considered to have a crisis if the loss it suffers over the `time_window` is larger than `large_loss`.

In [215]:
large_loss = -0.30 # large loss (a percentage number: large_loss = -0.30 represents a loss of -30%)

## Implementation

### Get the stock historical price data for the period under study

Use the yfinance module to download the daily data for the period. And store the result in a Pandas DataFrame.

In [216]:
df = yf.download(tickers=stock, period= period, interval='1d') # Download the data
df.drop(columns=['Open','High','Low','Adj Close','Volume'],inplace=True) # Drop unnecessary columns
df.insert(0, 'Company', stock_name) # Insert a column "Company" with the stock name in it
# Reset the index in order to make the "Date" column a normal column.
df.reset_index(inplace=True)
df.head() # Display the first rows of the data

[*********************100%***********************]  1 of 1 completed


,Date,Company,Close
0,1962-01-02,Boeing,0.823045
1,1962-01-03,Boeing,0.839506
2,1962-01-04,Boeing,0.831276
3,1962-01-05,Boeing,0.814815
4,1962-01-08,Boeing,0.816872


### Calculate Change over the `time_window` rolling period

Create a column to store the reference price at the beginning of the time window.

In [217]:
df['Reference Price'] = df.Close.shift(time_window)
df

,Date,Company,Close,Reference Price
0,1962-01-02,Boeing,0.823045,NaN
1,1962-01-03,Boeing,0.839506,NaN
2,1962-01-04,Boeing,0.831276,NaN
3,1962-01-05,Boeing,0.814815,NaN
4,1962-01-08,Boeing,0.816872,NaN
...,...,...,...,...
14653,2020-03-19,Boeing,97.709999,336.279999
14654,2020-03-20,Boeing,95.010002,330.380005
14655,2020-03-23,Boeing,105.620003,317.899994
14656,2020-03-24,Boeing,127.680000,304.140015


At the beginning of the data, we cannot get a reference point back the entire time window because we do not have the data before that point.

We will replace the missing values by the oldest possible price point available: the first historical price point available in the data.

In [218]:
df.fillna(value=df.iloc[0]['Close'],inplace=True)
df

,Date,Company,Close,Reference Price
0,1962-01-02,Boeing,0.823045,0.823045
1,1962-01-03,Boeing,0.839506,0.823045
2,1962-01-04,Boeing,0.831276,0.823045
3,1962-01-05,Boeing,0.814815,0.823045
4,1962-01-08,Boeing,0.816872,0.823045
...,...,...,...,...
14653,2020-03-19,Boeing,97.709999,336.279999
14654,2020-03-20,Boeing,95.010002,330.380005
14655,2020-03-23,Boeing,105.620003,317.899994
14656,2020-03-24,Boeing,127.680000,304.140015


Calculate the change column: last "Close" compared to "Reference Price" (the reference price being the price of the stock at the start of the `time_window`.

In [219]:
df['Change'] = (df['Close']-df['Reference Price']) / df['Reference Price']
df

,Date,Company,Close,Reference Price,Change
0,1962-01-02,Boeing,0.823045,0.823045,0.000000
1,1962-01-03,Boeing,0.839506,0.823045,0.020000
2,1962-01-04,Boeing,0.831276,0.823045,0.010000
3,1962-01-05,Boeing,0.814815,0.823045,-0.010000
4,1962-01-08,Boeing,0.816872,0.823045,-0.007500
...,...,...,...,...,...
14653,2020-03-19,Boeing,97.709999,336.279999,-0.709439
14654,2020-03-20,Boeing,95.010002,330.380005,-0.712422
14655,2020-03-23,Boeing,105.620003,317.899994,-0.667757
14656,2020-03-24,Boeing,127.680000,304.140015,-0.580193


### Find the worst crisis

To find the worst crisis, we are going to search for local minimums in the "Change" column. 

And then find those "Change" local minimums that are inferior to the `large_loss` parameter.

E.g.: Worst crisis of Boeing over a month (loss larger than -30%).

In [220]:
# Create a column with the local minimums (change smaller than the one before and the one after)
df['Local Min'] = df.Change[ (df.Change.shift(1)>df.Change) & (df.Change.shift(-1)>df.Change) ] 
df

,Date,Company,Close,Reference Price,Change,Local Min
0,1962-01-02,Boeing,0.823045,0.823045,0.000000,NaN
1,1962-01-03,Boeing,0.839506,0.823045,0.020000,NaN
2,1962-01-04,Boeing,0.831276,0.823045,0.010000,NaN
3,1962-01-05,Boeing,0.814815,0.823045,-0.010000,-0.010000
4,1962-01-08,Boeing,0.816872,0.823045,-0.007500,NaN
...,...,...,...,...,...,...
14653,2020-03-19,Boeing,97.709999,336.279999,-0.709439,NaN
14654,2020-03-20,Boeing,95.010002,330.380005,-0.712422,-0.712422
14655,2020-03-23,Boeing,105.620003,317.899994,-0.667757,NaN
14656,2020-03-24,Boeing,127.680000,304.140015,-0.580193,NaN


Address the fact that the very last price could be a local minimum as well.

If the very last price is smaller than the day before, then consider it as a local minimum as well.

In [221]:
if df.loc[df.index[-1],'Change'] < df.loc[df.index[-2],'Change']: # if the last change is smaller tham the day before  
    df.loc[df.index[-1],'Local Min'] = df.loc[df.index[-1],'Change'] # consider the last change as a local minimum
    
df

,Date,Company,Close,Reference Price,Change,Local Min
0,1962-01-02,Boeing,0.823045,0.823045,0.000000,NaN
1,1962-01-03,Boeing,0.839506,0.823045,0.020000,NaN
2,1962-01-04,Boeing,0.831276,0.823045,0.010000,NaN
3,1962-01-05,Boeing,0.814815,0.823045,-0.010000,-0.010000
4,1962-01-08,Boeing,0.816872,0.823045,-0.007500,NaN
...,...,...,...,...,...,...
14653,2020-03-19,Boeing,97.709999,336.279999,-0.709439,NaN
14654,2020-03-20,Boeing,95.010002,330.380005,-0.712422,-0.712422
14655,2020-03-23,Boeing,105.620003,317.899994,-0.667757,NaN
14656,2020-03-24,Boeing,127.680000,304.140015,-0.580193,NaN


Find out the worst crisis by selecting  the rows for which the Change has a Local Minimum inferior to the `large_loss` parameter defined at the top of the Notebook.

`df1`will be a smaller copy of the larger `df`. `df1` being will be used to ultimately represent the final output of that entire Notebook.

In [222]:
df1 = df[ df['Local Min'] < large_loss ].copy()
df1

,Date,Company,Close,Reference Price,Change,Local Min
2011,1970-02-05,Boeing,0.699588,1.020576,-0.314516,-0.314516
2977,1973-12-03,Boeing,0.465021,0.666667,-0.302469,-0.302469
2979,1973-12-05,Boeing,0.448560,0.658436,-0.318750,-0.318750
9996,2001-09-18,Boeing,33.139999,55.889999,-0.407050,-0.407050
9998,2001-09-20,Boeing,29.760000,55.130001,-0.460185,-0.460185
10002,2001-09-26,Boeing,34.290001,52.299999,-0.344359,-0.344359
10005,2001-10-01,Boeing,32.400002,53.060001,-0.389371,-0.389371
11775,2008-10-10,Boeing,41.799999,63.299999,-0.339652,-0.339652
11871,2009-03-02,Boeing,29.510000,42.310001,-0.302529,-0.302529
14648,2020-03-12,Boeing,154.839996,347.450012,-0.554353,-0.554353


It happens that we have several local minimums in the span of a `time_window`.

In order to avoid redundancy and keep only the worst loss point for each crisis, we will keep only the largest loss endured during the span of a given `time_window`.

The following function will use a `time_cursor` that will go iteratively at the start of each `time_window`, get the local minimums for that `time_window`, and keep only the worst date as being representative of the crisis during that `time_window`.

In [223]:
crisis_indexes = [] # initiate a list that will contain the crisis dates
time_cursor = df1.loc[df1.index[0],'Date'] # initiate a time cursor that will move at the start of each time_window to consider
print(f'time_cursor initiated at: {time_cursor}')
condition = df1['Date'] - time_cursor < pd.Timedelta(days=30)
time_window_df1 = df1[condition] # select crisis rows whose date are within 30 days of the time cursor

# Loop running as long as the cursor can be moved to a next time_window (i.e as long as there are dates moving forward)

while True:

    print('\nData for Time window being considered:\n')
    print( time_window_df1 )
    # get the date of the worst crisis during the time_window and append to the list of crisis dates
    crisis_indexes.append( time_window_df1['Change'].idxmin() ) 
    print(f'The crisis indexes so far are: {crisis_indexes}')

    # Try to get the next row after that time_window, and place the time_cursor there
    try:
        next_row = df1[ df1['Date'] > time_window_df1.loc[time_window_df1.index[-1],'Date'] ].head(1) # Try to get the next row after that time_window
        print('\nNext row will be:')
        print(next_row)
        time_cursor = next_row.loc[next_row.index[0],'Date'] # place the time_cursor at the date corresponding to that next row
        print(f'Moving the time_cursor to the start of the next time window to consider: {time_cursor}')
        # update the condition with new time_cursor (date after time_cursor, but within 30 days)
        condition = ( (df1['Date']-time_cursor) >= pd.Timedelta(days=0) ) & ( (df1['Date']-time_cursor) <= pd.Timedelta(days=30) )  
        time_window_df1 = df1[condition] # update crisis rows whose date are within 30 days of the time cursor
        
    # If no next row, we are at the end of the data and we can break the loop there
    except:
        print(f'There is not next date to which to move the the time cursor that is currently at {time_cursor}\nBREAK OUT OF THE LOOP')
        break



time_cursor initiated at: 1970-02-05 00:00:00

Data for Time window being considered:

           Date Company     Close  Reference Price    Change  Local Min
2011 1970-02-05  Boeing  0.699588         1.020576 -0.314516  -0.314516
The crisis indexes so far are: [2011]

Next row will be:
           Date Company     Close  Reference Price    Change  Local Min
2977 1973-12-03  Boeing  0.465021         0.666667 -0.302469  -0.302469
Moving the time_cursor to the start of the next time window to consider: 1973-12-03 00:00:00

Data for Time window being considered:

           Date Company     Close  Reference Price    Change  Local Min
2977 1973-12-03  Boeing  0.465021         0.666667 -0.302469  -0.302469
2979 1973-12-05  Boeing  0.448560         0.658436 -0.318750  -0.318750
The crisis indexes so far are: [2011, 2979]

Next row will be:
           Date Company      Close  Reference Price   Change  Local Min
9996 2001-09-18  Boeing  33.139999        55.889999 -0.40705   -0.40705
Moving the 

### We found the worst crisis dates:

In [224]:
crisis_indexes

[2011, 2979, 9998, 11775, 11871, 14654]

The data corresponding to those dates is:

In [225]:
df1 = df1.loc[crisis_indexes]
df1

,Date,Company,Close,Reference Price,Change,Local Min
2011,1970-02-05,Boeing,0.699588,1.020576,-0.314516,-0.314516
2979,1973-12-05,Boeing,0.448560,0.658436,-0.318750,-0.318750
9998,2001-09-20,Boeing,29.760000,55.130001,-0.460185,-0.460185
11775,2008-10-10,Boeing,41.799999,63.299999,-0.339652,-0.339652
11871,2009-03-02,Boeing,29.510000,42.310001,-0.302529,-0.302529
14654,2020-03-20,Boeing,95.010002,330.380005,-0.712422,-0.712422


### Find the date by which the stock comes back to its initial price

For a given crisis, we are going to retrieve the date by which the stock came back to the reference price from which it has fallen.

In [226]:
for index,row in df1.iterrows(): # Iterate through crisis dates
    # Try to get the first day after the crisis  where the stock closed higher than the Reference Price. 
    # If it exists (thus the Try)
    try:
        row_when_came_back = df.loc[(df['Date']>row['Date']) & (df['Close']>row['Reference Price'])].head(1)
        date_when_came_back = row_when_came_back.loc[row_when_came_back.index[0],'Date']
        df1.loc[index,'got back by'] = date_when_came_back
    except:
        print(f'Could not find a date by which the stock got back to its reference price for crisis at index: {index}')
        
df1    

Could not find a date by which the stock got back to its reference price for crisis at index: 14654


,Date,Company,Close,Reference Price,Change,Local Min,got back by
2011,1970-02-05,Boeing,0.699588,1.020576,-0.314516,-0.314516,1975-06-03
2979,1973-12-05,Boeing,0.448560,0.658436,-0.318750,-0.318750,1974-08-22
9998,2001-09-20,Boeing,29.760000,55.130001,-0.460185,-0.460185,2004-09-17
11775,2008-10-10,Boeing,41.799999,63.299999,-0.339652,-0.339652,2010-02-19
11871,2009-03-02,Boeing,29.510000,42.310001,-0.302529,-0.302529,2009-05-05
14654,2020-03-20,Boeing,95.010002,330.380005,-0.712422,-0.712422,NaT


And indicate the duration it took for the stock to come back to its reference price (to come back from the crisis)

create `delta` which is a series of Timedelta objects

In [227]:
# Using Timestamp only
time_delta = df1['got back by'] - df1["Date"]
time_delta

2011    1944 days
2979     260 days
9998    1093 days
11775    497 days
11871     64 days
14654         NaT
dtype: timedelta64[ns]

Get the days out of the time delta and translate it in years

In [228]:
delta = time_delta.apply(lambda x:x.days/365.0)
delta

2011     5.326027
2979     0.712329
9998     2.994521
11775    1.361644
11871    0.175342
14654         NaN
dtype: float64

In [229]:
df1['Duration (years)'] = delta
df1

,Date,Company,Close,Reference Price,Change,Local Min,got back by,Duration (years)
2011,1970-02-05,Boeing,0.699588,1.020576,-0.314516,-0.314516,1975-06-03,5.326027
2979,1973-12-05,Boeing,0.448560,0.658436,-0.318750,-0.318750,1974-08-22,0.712329
9998,2001-09-20,Boeing,29.760000,55.130001,-0.460185,-0.460185,2004-09-17,2.994521
11775,2008-10-10,Boeing,41.799999,63.299999,-0.339652,-0.339652,2010-02-19,1.361644
11871,2009-03-02,Boeing,29.510000,42.310001,-0.302529,-0.302529,2009-05-05,0.175342
14654,2020-03-20,Boeing,95.010002,330.380005,-0.712422,-0.712422,NaT,NaN


In [230]:
time_delta

2011    1944 days
2979     260 days
9998    1093 days
11775    497 days
11871     64 days
14654         NaT
dtype: timedelta64[ns]

Develop the duration into the exact number of years, months and days:

In [236]:
for index,row in df1.iterrows():
    print(row['Date'].date())
    row['Date'].date()
    my_relative_delta = relativedelta( row['Date'].date() , row['Date'].date() )
    print(my_relative_delta.days)
    df1.loc[index,'years'] = my_relative_delta.years
    df1.loc[index,'months'] = my_relative_delta.months
    df1.loc[index,'days'] = my_relative_delta.days
df1


1970-02-05
0
1973-12-05
0
2001-09-20
0
2008-10-10
0
2009-03-02
0
2020-03-20
0


,Date,Low,from High,Change,got back by,after (years),years,months,days
Crisis,,,,,,,,,
2011,1970-02-05,0.699588,1.020576,-0.314516,1975-06-03,5.326027,0.0,0.0,0.0
2979,1973-12-05,0.448560,0.658436,-0.318750,1974-08-22,0.712329,0.0,0.0,0.0
9998,2001-09-20,29.760000,55.130001,-0.460185,2004-09-17,2.994521,0.0,0.0,0.0
11775,2008-10-10,41.799999,63.299999,-0.339652,2010-02-19,1.361644,0.0,0.0,0.0
11871,2009-03-02,29.510000,42.310001,-0.302529,2009-05-05,0.175342,0.0,0.0,0.0
14654,2020-03-20,95.010002,330.380005,-0.712422,NaT,NaN,0.0,0.0,0.0


### Rename the columns to make it more readable as a final output

In [232]:
# Rename columns to make it more readable
df1.rename(columns={'Close': 'Low', 'Reference Price':'from High', 'Duration (years)':'after (years)'}, inplace=True)
# Drop unnecessary columns
df1.drop(columns=['Local Min','Company'],inplace=True)
# Rename the index to "Crisis"
df1.index.rename('Crisis',inplace=True)
df1

,Date,Low,from High,Change,got back by,after (years),years,months,days
Crisis,,,,,,,,,
2011,1970-02-05,0.699588,1.020576,-0.314516,1975-06-03,5.326027,0.0,0.0,0.0
2979,1973-12-05,0.448560,0.658436,-0.318750,1974-08-22,0.712329,0.0,0.0,0.0
9998,2001-09-20,29.760000,55.130001,-0.460185,2004-09-17,2.994521,0.0,0.0,0.0
11775,2008-10-10,41.799999,63.299999,-0.339652,2010-02-19,1.361644,0.0,0.0,0.0
11871,2009-03-02,29.510000,42.310001,-0.302529,2009-05-05,0.175342,0.0,0.0,0.0
14654,2020-03-20,95.010002,330.380005,-0.712422,NaT,NaN,0.0,0.0,0.0


Make the Change column readable as a percentage

# RESULT

Create a df2 copy of the df1 that will be formatted to be the ouptup for readability.

In [233]:
df2 = df1.copy()

In [234]:
df2["Change"] = df2["Change"] * 100

In [235]:
# Formatting the numbers
df2.index = df2.index.strftime("%Y-%m-%d") # Replace the index by formatted strings Y-d-m
df2.style.format({'Low':'${:.2f}','from High':'${:.2f}','Change':'{:.2f}%','got back by':'{}','after (years)':'{:.2f}',
                  'years':'{:.0f}','months':'{:.0f}','days':'{:.0f}'},na_rep="-")\
        .bar(subset=['Change'],color='#E98888',align='mid')\
        .bar(subset=['after (years)'],color='#88ABE9',align='mid')\
        .set_caption(f"Company: {stock_name}")

AttributeError: 'Int64Index' object has no attribute 'strftime'

In [ ]:
#df.to_csv('BA.csv')

# ANNEX (Plot)

## Plot the period where the stock suffered a crisis and it took the MOST time to come back

Find the row corresponding to the period suffered a crisis and it took the longer to come back from it:

In [ ]:
df1.loc[ df1["after (years)"].idxmax() ]

In [ ]:
df1["after (years)"].idxmax()

In [ ]:
df1.loc[ df1["after (years)"].idxmax() ,"got back by"]

In [ ]:
df.loc[ df1["after (years)"].idxmax() - DateOffset(days=30) : df1.loc[ df1["after (years)"].idxmax() + Timedelta(days=30) ,"got back by"] ]

## Plot the period where the stock suffered a crisis and it took the LESS time to come back

In [ ]:
# Will allow us to embed images in the notebook
%matplotlib inline

register_matplotlib_converters()

# Create a figure containing a single axes
fig, ax = plt.subplots()

# Draw on the axes
ax.plot(df.index.values,df['Change'])

plt.show()